In [43]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

#

import matplotlib.pyplot as plt

from pandasgui import show

#

import warnings

In [44]:
warnings.filterwarnings( 'ignore' )

In [45]:
gmID_list = cfs.list_whitelisted_gmIDs()

In [46]:
topic_list = cfs.list_topics()

print( topic_list )

['/apollo/canbus/chassis', '/apollo/drive/event', '/apollo/sensor/gnss/best/pose', '/apollo/perception/traffic/light']


In [47]:
def DistanceToNearestDisengagement( time_sorted_chassis_df ):

    chassis_time_array = np.array( time_sorted_chassis_df[ 'time' ] ) * 1e-9 # seconds

    chassis_speedMps_array = np.array( time_sorted_chassis_df[ 'speedMps' ] ) # meters/second

    chassis_TernaryDrivingModeTransition_array = np.array( time_sorted_chassis_df[ 'TernaryDrivingModeTransition' ] )

    #

    chassis_deltatime_array = np.diff( chassis_time_array )

    chassis_deltatime_array = np.insert( chassis_deltatime_array, 0, chassis_deltatime_array[ 0 ] )

    #

    chassis_deltadistance_array = chassis_deltatime_array * chassis_speedMps_array

    chassis_distance_array = np.cumsum( chassis_deltadistance_array )

    #

    chassis_disengagement_indexes = np.where( chassis_TernaryDrivingModeTransition_array == -1 )

    chassis_disengagement_distance_array = chassis_distance_array[ chassis_disengagement_indexes ]

    #

    if ( len( chassis_disengagement_indexes[ 0 ] ) == 0 ):

        time_sorted_chassis_df[ 'DistanceToNearestDisengagement' ] = [ np.nan for index in chassis_time_array ]

        return None

    #

    distance_to_disengagement_array_list = []

    for disengagement_distance in chassis_disengagement_distance_array:

        distance_to_disengagement_array = chassis_distance_array - disengagement_distance

        distance_to_disengagement_array_list.append( distance_to_disengagement_array )

    distance_to_disengagement_array_matrix = np.vstack( distance_to_disengagement_array_list )

    chassis_DistanceToNearestDisengagement_list = []
    
    for colnum in range( distance_to_disengagement_array_matrix.shape[ 1 ] ):

        col = distance_to_disengagement_array_matrix[ :, colnum ]

        abs_col = np.abs( col )

        min_abs_col_index = np.where( abs_col == np.min( abs_col ) )

        DistanceToNearestDisengagement = col[ min_abs_col_index ][ 0 ]

        chassis_DistanceToNearestDisengagement_list.append( DistanceToNearestDisengagement )

    time_sorted_chassis_df[ 'DistanceToNearestDisengagement' ] = chassis_DistanceToNearestDisengagement_list

In [48]:
chassis_df_list = []

for index, gmID in enumerate( gmID_list ):

    temp_gmID_chassis_df = cfs.retrieve_gmID_topic( gmID_list[ index ], '/apollo/canbus/chassis' )

    temp_gmID_chassis_df = temp_gmID_chassis_df.sort_values( 'time' )
    
    cfs.BinaryDrivingMode( temp_gmID_chassis_df )

    cfs.TernaryDrivingModeTransition( temp_gmID_chassis_df )

    DistanceToNearestDisengagement( temp_gmID_chassis_df )

    chassis_df_list.append( temp_gmID_chassis_df )

    print( index )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
